In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI：SDK BigQuery 自定义容器训练

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/sdk/SDK_BigQuery_Custom_Container_Training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/sdk/SDK_BigQuery_Custom_Container_Training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/sdk/SDK_BigQuery_Custom_Container_Training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

### 概述

这个笔记本使用bigquery数据集创建自定义容器，它将训练容器，并创建、训练和部署模型以执行预测。

### 目标

在这个笔记本中，您将学习如何使用Vertex AI实验来：

* 记录管道作业
* 比较不同的管道作业

涵盖的步骤包括：

* 规范化训练组件
* 构建训练模型
* 运行多个管道作业并记录它们的结果
* 训练模型进行预测

数据集

本教程使用的数据集是[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)中的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)。该数据集不需要任何特征工程。在本教程中使用的数据集版本存储在公共云存储桶中。训练好的模型可以预测出三个品种的鸢尾花物种：山鸢尾、弗吉尼亚鸢尾或变色鸢尾。

### 费用

本教程使用了Google Cloud的可计费组件：

- Vertex AI
- 云存储

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)，[Bigquery价格](https://cloud.google.com/bigquery/pricing/)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据您的预计使用量生成费用估算。

建立您的本地开发环境

**如果您正在使用Colab或Google Cloud笔记本**，您的环境已经满足运行此笔记本的所有要求。 您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简化的一套说明：

1. [安装和初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端窗口的命令行中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端窗口的命令行中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

# 确保以下API已启用：
- [BigQuery](https://console.cloud.google.com/apis/library/bigquery.googleapis.com?q=BigQuery)
- [Cloudbuild](https://console.cloud.google.com/apis/library/cloudbuild.googleapis.com?q=Cloudbuild)
- [Container Registry](https://console.cloud.google.com/apis/library/containerregistry.googleapis.com?q=container%20registry)

安装额外的包

在您的笔记本环境中安装未安装的额外包依赖项，如XGBoost、AdaNet或TensorFlow Hub。使用每个包的最新正式版本。

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

重新启动内核

在安装附加包之后，您需要重新启动笔记本内核，这样它才能找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在下面的单元格中输入您的项目 ID 和 GCS 存储桶。

在下面的单元格中输入您的项目 ID，然后运行该单元格，以确保 Cloud SDK 在这个笔记本中对所有命令使用正确的项目。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
import os

PROJECT_ID = ""
# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，请在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    print("Project ID: ", PROJECT_ID)

### 认证您的谷歌云账户

**如果您正在使用 Vertex AI Workbench Notebooks**，您的环境已经认证。请跳过此步骤。

**如果您正在使用 Colab**，请运行下面的单元格，并按照提示进行身份验证，通过oAuth认证您的账户。

**否则**，请按照以下步骤操作：

1. 在云控制台中，转到[**创建服务账号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务账号**。

3. 在**服务账号名称**字段中输入名称，然后点击**创建**。

4. 在**授予此服务账号访问项目**部分，点击**角色**下拉列表。在过滤框中输入"Vertex AI"，选择**Vertex AI管理员**。在过滤框中输入" Storage Object Admin"，选择**存储对象管理员**。

5. 点击*创建*。一个包含您密钥的JSON文件将下载到您的本地环境。

6. 在下方单元格中将您的服务账号密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，并运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

UUID

如果您正在进行现场教程会话，您可能正在使用一个共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您可以为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

创建存储桶

**无论您使用哪种笔记本环境，都需要执行以下步骤。**

使用 Cloud SDK 提交训练作业时，您需要将包含训练代码的 Python 包上传到 Cloud 存储桶中。Vertex AI 会从这个包中运行代码。在本教程中，Vertex AI 还会将作业产生的训练模型保存在同一个存储桶中。使用这个模型工件，您可以创建 Vertex AI 模型和端点资源，以便提供在线预测。

在下方设置您的 Cloud 存储桶的名称。它必须在所有 Cloud 存储桶中是唯一的。

您还可以更改“REGION”变量，该变量会在此笔记本的其余部分中使用。我们建议您[选择一个支持 Vertex AI 服务的地区](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + UUID

if REGION == "[your-region]":
    REGION = "us-central1"

只有如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查其内容来验证对您的云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

复制bigquery鸢尾花数据集

您创建了一个BigQuery数据集，并将BigQuery的公共鸢尾花表复制到该数据集中。有关此数据集的更多信息，请访问：https://archive.ics.uci.edu/ml/datasets/iris

### 制作 BQ 数据集

In [ ]:
import os

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
!bq mk {PROJECT_ID}:ml_datasets

### 复制bigquery-public-data.ml_datasets.iris

In [ ]:
!bq cp -n --project_id={PROJECT_ID} bigquery-public-data:ml_datasets.iris {PROJECT_ID}:ml_datasets.iris 

# 创建训练容器
我们将创建一个目录，并将所有容器构建产物写入该文件夹中。

In [ ]:
CONTAINER_ARTIFACTS_DIR = "demo-container-artifacts"

!mkdir {CONTAINER_ARTIFACTS_DIR}

创建Cloudbuild YAML

In [ ]:
cloudbuild_yaml = """steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', 'gcr.io/{PROJECT_ID}/test-custom-container', '.' ]
images: ['gcr.io/{PROJECT_ID}/test-custom-container']""".format(
    PROJECT_ID=PROJECT_ID
)

with open(f"{CONTAINER_ARTIFACTS_DIR}/cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

写这个 Dockerfile

In [ ]:
%%writefile {CONTAINER_ARTIFACTS_DIR}/Dockerfile

# Specifies base image and tag
FROM gcr.io/google-appengine/python
WORKDIR /root

# Installs additional packages
RUN pip3 install tensorflow tensorflow-io pyarrow

# Copies the trainer code to the docker image.
COPY test_script.py /root/test_script.py

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python3", "test_script.py"]

### 编写入口脚本调用训练器

入口脚本训练和验证数据，并且编译模型。

In [ ]:
%%writefile {CONTAINER_ARTIFACTS_DIR}/test_script.py

from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf
from tensorflow import feature_column
import os

training_data_uri = os.environ["AIP_TRAINING_DATA_URI"]
validation_data_uri = os.environ["AIP_VALIDATION_DATA_URI"]
test_data_uri = os.environ["AIP_TEST_DATA_URI"]
data_format = os.environ["AIP_DATA_FORMAT"]

def caip_uri_to_fields(uri):
    uri = uri[5:]
    project, dataset, table = uri.split('.')
    return project, dataset, table

feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

target_name = 'species'

def transform_row(row_dict):
  # Trim all string tensors
  trimmed_dict = { column:
                  (tf.strings.strip(tensor) if tensor.dtype == 'string' else tensor) 
                  for (column,tensor) in row_dict.items()
                  }
  target = trimmed_dict.pop(target_name)

  target_float = tf.cond(tf.equal(tf.strings.strip(target), 'versicolor'), 
                 lambda: tf.constant(1.0),
                 lambda: tf.constant(0.0))
  return (trimmed_dict, target_float)

def read_bigquery(project, dataset, table):
  tensorflow_io_bigquery_client = BigQueryClient()
  read_session = tensorflow_io_bigquery_client.read_session(
      "projects/" + project,
      project, table, dataset,
      feature_names + [target_name],
      [dtypes.float64] * 4 + [dtypes.string],
      requested_streams=2)

  dataset = read_session.parallel_read_rows()
  transformed_ds = dataset.map(transform_row)
  return transformed_ds

BATCH_SIZE = 16

training_ds = read_bigquery(*caip_uri_to_fields(training_data_uri)).shuffle(10).batch(BATCH_SIZE)
eval_ds = read_bigquery(*caip_uri_to_fields(validation_data_uri)).batch(BATCH_SIZE)
test_ds = read_bigquery(*caip_uri_to_fields(test_data_uri)).batch(BATCH_SIZE)

feature_columns = []

# numeric cols
for header in feature_names:
  feature_columns.append(feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Dense = tf.keras.layers.Dense
model = tf.keras.Sequential(
  [
    feature_layer,
      Dense(16, activation=tf.nn.relu),
      Dense(8, activation=tf.nn.relu),
      Dense(4, activation=tf.nn.relu),
      Dense(1, activation=tf.nn.sigmoid),
  ])

# Compile Keras model
model.compile(
    loss='binary_crossentropy', 
    metrics=['accuracy'],
    optimizer='adam')

model.fit(training_ds, epochs=5, validation_data=eval_ds)

print(model.evaluate(test_ds))

tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])

构建容器

In [ ]:
!gcloud builds submit --project={PROJECT_ID} --config {CONTAINER_ARTIFACTS_DIR}/cloudbuild.yaml {CONTAINER_ARTIFACTS_DIR}

# 运行自定义容器培训

初始化Python的Vertex AI SDK

初始化Vertex AI的客户端

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

从前面复制的 iris BigQuery 表中创建一个托管的表格数据集。使用的参数是 BigQuery 的公共 iris 数据集。# 从 bigquery 数据集创建管理的表格数据集

In [ ]:
ds = aiplatform.TabularDataset.create(
    display_name="bq_iris_dataset", bq_source=f"bq://{PROJECT_ID}.ml_datasets.iris"
)

启动训练工作以创建模型

我们将使用上面构建的容器来训练模型。要训练模型，您可以使用CustomContainer TrainingJob方法，参数为Container Image和Container_uri。

In [ ]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="train-bq-iris",
    container_uri=f"gcr.io/{PROJECT_ID}/test-custom-container:latest",
    model_serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest",
)
model = job.run(
    ds,
    replica_count=1,
    model_display_name="bq-iris-model",
    bigquery_destination=f"bq://{PROJECT_ID}",
)

部署模型

部署您的模型，然后等待模型完成部署后再继续预测。对于预测部署方法，需要传入 machine_type 参数。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

做一个预测

端点预测方法根据长度和宽度特征参数发布预测。

In [ ]:
prediction = endpoint.predict(
    [{"sepal_length": 5.1, "sepal_width": 2.5, "petal_length": 3.0, "petal_width": 1.1}]
)

print(prediction)

清理工作

要清理此项目中使用的所有谷歌云资源，您可以[删除您用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 管道
- 端点
- 云存储桶

In [ ]:
delete_pipeline = True
delete_endpoint = True


if delete_pipeline:
    job.delete()

    if delete_endpoint and "DISPLAY_NAME" in globals():
        endpoints = aip.Endpoint.list(
            filter=f"display_name={DISPLAY_NAME}_endpoint", order_by="create_time"
        )
        if endpoints:
            endpoint = endpoints[0]
            endpoint.undeploy_all()
            aip.Endpoint.delete(endpoint.resource_name)
            print("Deleted endpoint:", endpoint)


# Delete bucket
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}